# [1] Загрузка данных 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Загружаем данные
data = pd.read_csv('marketing_campaign.csv', sep='\t')

print(f"Размер датасета: {data.shape}")
print(f"Колонки: {data.columns.tolist()}")


Размер датасета: (2240, 29)
Колонки: ['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response']


# [2] Определение типа задачи

In [3]:
# -- Повторная информация о целевой переменной --
target = 'Response'
print(f"Целевая переменная: {target}")
print(f"Уникальные значения: {data[target].unique()}")
print(f"Количество уникальных значений: {data[target].nunique()}")
print(f"Распределение: {data[target].value_counts().to_dict()}")


Целевая переменная: Response
Уникальные значения: [1 0]
Количество уникальных значений: 2
Распределение: {0: 1906, 1: 334}


Задача бинарной классификации. Значения target Response: 0 или 1 - это "категории", а не "числовые значения". 

# [3] Предобработку данных

### A: Разделисть выборку на тренировочную (train) и тестовую (test)

In [4]:
# --- 1. ОТДЕЛЯЕМ ПРИЗНАКИ И ЦЕЛЬ ---

# Целевая переменная
y = data['Response']

# Признаки (убираем целевой столбец и явные служебные поля)
X = data.drop(columns=['Response', 'ID', 'Z_CostContact', 'Z_Revenue'])

print("Форма X:", X.shape)
print("Форма y:", y.shape)


# ---  train/test  ---
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,      # 20% в тест
    random_state=42,    # фиксируем для воспроизводимости
    stratify=y          # сохраняем пропорции классов 0/1
)

print("Train X:", X_train.shape)
print("Test  X:", X_test.shape)
print("Распределение классов в train:")
print(y_train.value_counts(normalize=True))
print("Распределение классов в test:")
print(y_test.value_counts(normalize=True))


Форма X: (2240, 25)
Форма y: (2240,)
Train X: (1792, 25)
Test  X: (448, 25)
Распределение классов в train:
Response
0    0.851004
1    0.148996
Name: proportion, dtype: float64
Распределение классов в test:
Response
0    0.850446
1    0.149554
Name: proportion, dtype: float64


### B: Проверить пропуски в данных

In [10]:
# ---  АНАЛИЗ ПРОПУСКОВ ---

print("Пропуски в train:")
print(X_train.isnull().sum())

print("\nПропуски в test:")
print(X_test.isnull().sum())

# Разделяем признаки по типам
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()

print("\nЧисловые признаки:", num_cols)
print("Категориальные признаки:", cat_cols)

# --- ЗАПОЛНЕНИЕ ПРОПУСКОВ ---

# 1. Числовые: заполняем медианой по train
from sklearn.impute import SimpleImputer

num_imputer = SimpleImputer(strategy='median')
num_imputer.fit(X_train[num_cols])          # считаем медианы только на train

X_train[num_cols] = num_imputer.transform(X_train[num_cols])
X_test[num_cols]  = num_imputer.transform(X_test[num_cols])

# 2. Категориальные заполняем модой (most_frequent)
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_imputer.fit(X_train[cat_cols])

X_train[cat_cols] = cat_imputer.transform(X_train[cat_cols])
X_test[cat_cols]  = cat_imputer.transform(X_test[cat_cols])

# Проверяем, что пропусков больше нет
print("\nПосле импутации, пропуски в train:")
print(X_train.isnull().sum())
print("\nПосле импутации, пропуски в test:")
print(X_test.isnull().sum())


Пропуски в train:
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
dtype: int64

Пропуски в test:
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
M

### C: Отнормировать численные переменные (StandardScaler)


Учим скейлер только на train, применяем и к train, и к test, и трогаем только числовые признаки (num_cols)

In [17]:
from sklearn.preprocessing import StandardScaler

# создаём скейлер
scaler = StandardScaler()

# считаем параметры (среднее и std) ТОЛЬКО по train
scaler.fit(X_train[num_cols])

# применяем к train и test
X_train[num_cols] = scaler.transform(X_train[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

X_train[num_cols].head()


,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain
1090,0.342139,1.294130,-0.828060,-0.925853,0.444943,1.891312,-0.183717,3.021240,-0.229925,1.645925,...,0.659757,1.195745,0.383416,-1.361588,-0.283141,-0.281989,3.636237,-0.264293,-0.114025,-0.10352
15,-1.943388,1.177270,-0.828060,-0.925853,-0.895179,2.093752,-0.108461,-0.227785,0.387369,0.993932,...,1.012678,1.195745,1.923956,-0.955063,-0.283141,-0.281989,3.636237,3.783681,-0.114025,-0.10352
873,1.019333,-0.373888,1.026794,0.899370,-0.482834,-0.758265,-0.334230,-0.486635,-0.248081,-0.285907,...,-0.399005,-0.567565,-0.540909,0.671039,-0.283141,-0.281989,-0.275010,-0.264293,-0.114025,-0.10352
610,0.003543,-0.969628,-0.828060,-0.925853,0.857289,-0.600481,-0.334230,-0.401839,-0.320703,-0.213464,...,-0.399005,-0.567565,0.075308,0.264514,-0.283141,-0.281989,-0.275010,-0.264293,-0.114025,-0.10352
657,-0.927598,-0.984485,-0.828060,-0.925853,1.475807,-0.767196,0.518675,-0.508950,-0.302548,-0.165168,...,-0.399005,-0.567565,-0.232800,0.264514,-0.283141,-0.281989,-0.275010,-0.264293,-0.114025,-0.10352


### D: Закодировать категориальные признаки по одной из стратегий.

In [19]:
from sklearn.preprocessing import OneHotEncoder

cat_cols = ['Education', 'Marital_Status']  # Dt_Customer пока не трогаем

# ВАЖНО: используем sparse_output вместо sparse
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

ohe.fit(X_train[cat_cols])

X_train_cat = ohe.transform(X_train[cat_cols])
X_test_cat  = ohe.transform(X_test[cat_cols])

ohe_feature_names = ohe.get_feature_names_out(cat_cols)

X_train_cat = pd.DataFrame(X_train_cat, columns=ohe_feature_names, index=X_train.index)
X_test_cat  = pd.DataFrame(X_test_cat,  columns=ohe_feature_names, index=X_test.index)

X_train_num = X_train.drop(columns=cat_cols + ['Dt_Customer'])
X_test_num  = X_test.drop(columns=cat_cols + ['Dt_Customer'])

X_train_final = pd.concat([X_train_num, X_train_cat], axis=1)
X_test_final  = pd.concat([X_test_num,  X_test_cat],  axis=1)

X_train_final.head()


,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,Education_Master,Education_PhD,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
1090,0.342139,1.294130,-0.828060,-0.925853,0.444943,1.891312,-0.183717,3.021240,-0.229925,1.645925,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15,-1.943388,1.177270,-0.828060,-0.925853,-0.895179,2.093752,-0.108461,-0.227785,0.387369,0.993932,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
873,1.019333,-0.373888,1.026794,0.899370,-0.482834,-0.758265,-0.334230,-0.486635,-0.248081,-0.285907,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
610,0.003543,-0.969628,-0.828060,-0.925853,0.857289,-0.600481,-0.334230,-0.401839,-0.320703,-0.213464,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
657,-0.927598,-0.984485,-0.828060,-0.925853,1.475807,-0.767196,0.518675,-0.508950,-0.302548,-0.165168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# [4] Обучить на тренировочном множестве

### A: Линейную модель ((LogisticRegression, LinearRegression))

In [20]:
from sklearn.linear_model import LogisticRegression

# 1. Создаём модель логистической регрессии
log_reg = LogisticRegression(
    max_iter=1000,      # увеличиваем число итераций, чтобы точно сошлась
    random_state=42
)

# 2. Обучаем на тренировочных данных
log_reg.fit(X_train_final, y_train)

# 3. Делаем предсказания (пока просто, без метрик)
y_train_pred_lr = log_reg.predict(X_train_final)
y_test_pred_lr  = log_reg.predict(X_test_final)

# На шаге 5 будем уже считать accuracy, f1, roc_auc и т.п.
y_train_pred_lr[:10], y_test_pred_lr[:10]


(array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

### B: Дереянную модель (DecisionTreeClassifier, DecisionTreeRegressor)

In [22]:
from sklearn.tree import DecisionTreeClassifier

# DecisionTreeClassifier (разные глубины)

best_dt = None          # лучшую модель дерева
best_dt_score = 0       # лучшее значение accuracy на тесте

# Перебираем разные варианты глубины дерева
for depth in [3, 5, 7, 10, 15]:
    # 1. Создаём дерево с заданной максимальной глубиной
    dt = DecisionTreeClassifier(
        max_depth=depth,    # ограничиваем глубину дерева
        random_state=42     # для воспроизводимости
    )
    
    # 2. Обучаем дерево на тренировочных данных
    dt.fit(X_train_final, y_train)
    
    # 3. Оцениваем качество на тестовой выборке
    # .score для классификатора = accuracy (доля верно предсказанных объектов)
    test_score = dt.score(X_test_final, y_test)
    
    # 4. Печатаем текущий результат
    print(f"max_depth={depth}: test_accuracy={test_score:.4f}")
    
    # 5. Если это дерево лучше предыдущего, запоминаем его
    if test_score > best_dt_score:
        best_dt_score = test_score
        best_dt = dt

# После цикла best_dt — лучшее дерево, best_dt_score — его accuracy на тесте
print(f"✓ DecisionTree обучена (best depth={best_dt.max_depth}, test_accuracy={best_dt_score:.4f})")


max_depth=3: test_accuracy=0.8616
max_depth=5: test_accuracy=0.8638
max_depth=7: test_accuracy=0.8594
max_depth=10: test_accuracy=0.8594
max_depth=15: test_accuracy=0.8371
✓ DecisionTree обучена (best depth=5, test_accuracy=0.8638)


### С: K-ближайших соседей (KNeighborsClassifier, KNeighborsRegressor) 

In [23]:
from sklearn.neighbors import KNeighborsClassifier

best_knn = None
best_knn_score = 0

# Перебираем разные значения k 
for k in [3, 5, 7, 9, 15]:
    knn = KNeighborsClassifier(
        n_neighbors=k,   # сколько ближайших соседей учитываем
        weights='distance',  
        metric='minkowski',  # по умолчанию евклидово расстояние (p=2)
        p=2
    )
    
    # Обучаем KNN: на самом деле он просто запоминает обучающую выборку
    knn.fit(X_train_final, y_train)
    
    # Оцениваем accuracy на тесте
    test_score = knn.score(X_test_final, y_test)
    print(f"k={k}: test_accuracy={test_score:.4f}")
    
    # Сохраняем лучшую модель
    if test_score > best_knn_score:
        best_knn_score = test_score
        best_knn = knn

print(f"KNN обучен (best k={best_knn.n_neighbors}, test_accuracy={best_knn_score:.4f})")

k=3: test_accuracy=0.8906
k=5: test_accuracy=0.8795
k=7: test_accuracy=0.8750
k=9: test_accuracy=0.8795
k=15: test_accuracy=0.8884
KNN обучен (best k=3, test_accuracy=0.8906)


### D: Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# [4.4] RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,   # число деревьев в лесу
    random_state=42,
    n_jobs=-1           # использовать все ядра CPU
)

# Обучаем лес на тренировочных данных
rf.fit(X_train_final, y_train)

# Предсказанные классы (0/1)
y_train_pred_rf = rf.predict(X_train_final)
y_test_pred_rf  = rf.predict(X_test_final)

# Предсказанные вероятности класса 1 (для ROC-AUC и пр.)
y_train_proba_rf = rf.predict_proba(X_train_final)[:, 1]
y_test_proba_rf  = rf.predict_proba(X_test_final)[:, 1]

print("RandomForest обучена (100 деревьев)")


✓ RandomForest обучена (100 деревьев)


# [5] Посчитайте метрики на train и test

In [27]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

results = {}

# 1. LogisticRegression (log_reg уже обучена ранее)
results['LogisticRegression'] = {
    'model': log_reg,
    'train_pred':  log_reg.predict(X_train_final),
    'test_pred':   log_reg.predict(X_test_final),
    'train_proba': log_reg.predict_proba(X_train_final)[:, 1],
    'test_proba':  log_reg.predict_proba(X_test_final)[:, 1],
}

# 2. Decision Tree (best_dt из цикла по max_depth)
results['DecisionTree'] = {
    'model': best_dt,
    'train_pred':  best_dt.predict(X_train_final),
    'test_pred':   best_dt.predict(X_test_final),
    'train_proba': best_dt.predict_proba(X_train_final)[:, 1],
    'test_proba':  best_dt.predict_proba(X_test_final)[:, 1],
}

# 3. KNN (best_knn из цикла по k)
results['KNN'] = {
    'model': best_knn,
    'train_pred':  best_knn.predict(X_train_final),
    'test_pred':   best_knn.predict(X_test_final),
    'train_proba': best_knn.predict_proba(X_train_final)[:, 1],
    'test_proba':  best_knn.predict_proba(X_test_final)[:, 1],
}

# 4. Random Forest (у тебя просто rf)
results['RandomForest'] = {
    'model': rf,
    'train_pred':  rf.predict(X_train_final),
    'test_pred':   rf.predict(X_test_final),
    'train_proba': rf.predict_proba(X_train_final)[:, 1],
    'test_proba':  rf.predict_proba(X_test_final)[:, 1],
}

# === Расчёт метрик ===

metrics_results = {}

for model_name in ['LogisticRegression', 'DecisionTree', 'KNN', 'RandomForest']:
    train_pred  = results[model_name]['train_pred']
    test_pred   = results[model_name]['test_pred']
    train_proba = results[model_name]['train_proba']
    test_proba  = results[model_name]['test_proba']
    
    train_acc = accuracy_score(y_train, train_pred)
    test_acc  = accuracy_score(y_test,  test_pred)
    
    train_auc = roc_auc_score(y_train, train_proba)
    test_auc  = roc_auc_score(y_test,  test_proba)
    
    train_f1 = f1_score(y_train, train_pred)
    test_f1  = f1_score(y_test,  test_pred)
    
    metrics_results[model_name] = {
        'train_acc': train_acc,
        'test_acc':  test_acc,
        'train_auc': train_auc,
        'test_auc':  test_auc,
        'train_f1':  train_f1,
        'test_f1':   test_f1
    }

metrics_df = pd.DataFrame.from_dict(metrics_results, orient='index')
print(metrics_df.round(3))


                    train_acc  test_acc  train_auc  test_auc  train_f1  \
LogisticRegression      0.895     0.884      0.884     0.892     0.559   
DecisionTree            0.908     0.864      0.839     0.742     0.606   
KNN                     0.995     0.891      1.000     0.768     0.983   
RandomForest            0.995     0.886      1.000     0.880     0.983   

                    test_f1  
LogisticRegression    0.490  
DecisionTree          0.384  
KNN                   0.533  
RandomForest          0.427  


# [6] Сравните метрики

Лучшая модель
С точки зрения баланса метрик на тесте лучше всего выглядит LogisticRegression: у неё высокая test‑accuracy (0.884), самый высокий ROC‑AUC на тесте (0.892) и неплохой F1 (0.490) при явном дисбалансе классов.​
KNN и RandomForest показывают чуть выше accuracy, но заметно хуже ROC‑AUC и F1, особенно дерево решений сильно проседает по ROC‑AUC и F1 на тесте.​

Переобучение

У дерева: train_acc 0.908 против test_acc 0.864, train_auc 0.839 против test_auc 0.742, train_f1 0.606 против test_f1 0.384 — модель явно сильно лучше запоминает train, чем обобщает на test → существенное переобучение.​

У KNN и RandomForest: train_acc и train_f1 почти 1.0, ROC‑AUC на train 1.0, а на тесте accuracy и F1 заметно ниже → тоже явное переобучение (модели почти идеально описали обучающую выборку).​

У логистической регрессии разница train/test по accuracy, ROC‑AUC и F1 небольшая → минимальное переобучение, модель наиболее устойчива.​

Недообучение
Недообучение — когда и на train, и на test показатели низкие.​

У логистической регрессии train_acc ~0.895 и train_auc ~0.884 — это довольно высокие значения, так что явного недообучения нет, скорее разумный компромисс между bias и variance.​

Дерево/лес/KNN имеют практически идеальные метрики на train, так что там недообучения нет; проблема именно в переобучении (variance), а не в недостаточной сложности модели.​

Как улучшить метрики моделей

Для дерева:

усилить регуляризацию: уменьшить max_depth, увеличить min_samples_leaf, min_samples_split, использовать max_features < 1.0; это снизит переобучение и должно поднять test‑ROC‑AUC и test‑F1.​

Для RandomForest:

аккуратно ограничить глубину деревьев (max_depth, min_samples_leaf), поиграть с n_estimators (200–500), возможно уменьшить max_features для большего разнообразия деревьев.​

Для KNN:

увеличить n_neighbors (например, 9–31), проверить оба варианта weights='uniform' и 'distance'; иногда при большем k модель меньше переобучается и ROC‑AUC/F1 на тесте растут.​

Для логистической регрессии:

поиграть с силой регуляризации C (меньше C — сильнее регуляризация, больше C — модель гибче) и типом регуляризации (penalty='l1'/'l2'); также можно попробовать добавить новые осмысленные признаки (например, из дат, отношений расходов и дохода), что часто даёт больший выигрыш, чем чистый тюнинг гиперпараметров.